 # ACE Next — Interactive Demo



 This notebook walks through the refactored `ace_next` pipeline.

 It covers:



 1. **Runners** — `ACE` (full pipeline) and `TraceAnalyser` (learning-only)

 2. **Steps** — individual pipeline steps and `learning_tail()`

 3. **Manual pipeline construction** — composing steps by hand

 4. **Custom environments** — writing your own evaluator

 5. **Checkpointing & deduplication** — production features

 6. **Observability with Opik** — pipeline traces and LLM cost tracking

 7. **Skillbook persistence** — save / reload

 8. **TraceAnalyser** — learning from pre-recorded traces



 **Requirements:** `uv sync` from the repo root.

 Set your LLM API key before running:

 ```bash

 export OPENAI_API_KEY="sk-..."

 ```

 ## 1. Setup & Imports

In [1]:
import os
import sys
import logging
import tempfile
from pathlib import Path

import nest_asyncio

nest_asyncio.apply()

# Silence LiteLLM's verbose logging so notebook output stays clean
logging.getLogger("LiteLLM").setLevel(logging.WARNING)
logging.getLogger("LiteLLM Router").setLevel(logging.WARNING)
logging.getLogger("LiteLLM Proxy").setLevel(logging.WARNING)

# Also suppress litellm's own set_verbose flag
try:
    import litellm
    litellm.set_verbose = False
except ImportError:
    pass

# Ensure the project root is on sys.path so `ace`, `ace_next`, and `pipeline`
# are importable regardless of where the notebook kernel starts.
_here = Path(__file__).resolve().parent if "__file__" in dir() else Path.cwd()
_root = _here
for _p in [_here] + list(_here.parents):
    if (_p / "pipeline" / "__init__.py").exists():
        _root = _p
        break
sys.path.insert(0, str(_root))

from dotenv import load_dotenv

load_dotenv(_root / ".env")

print(f"Project root: {_root}")
print("Setup OK")

Project root: /home/david/Desktop/projects/Kayba/agentic-context-engine
Setup OK


 ## 2. Core Imports



 Everything lives in `ace_next` — fully self-contained, zero cross-imports.

In [2]:
from ace_next import (
    # Runners
    ACE,
    TraceAnalyser,
    # Role implementations
    Agent,
    Reflector,
    SkillManager,
    # LLM providers
    LiteLLMClient,
    # Core types
    Sample,
    Skillbook,
    SimpleEnvironment,
    TaskEnvironment,
    EnvironmentResult,
    AgentOutput,
    # Context types
    ACEStepContext,
    SkillbookView,
)

print("All imports OK")


/home/david/Desktop/projects/Kayba/agentic-context-engine/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All imports OK


 ## 3. Configure the LLM Client



 We use LiteLLM which supports 100+ providers. Swap the model string

 for any provider: `gpt-4o-mini`, `claude-sonnet-4-5-20250929`,

 `bedrock/us.anthropic.claude-haiku-4-5-20251001-v1:0`, etc.

In [3]:
MODEL = os.getenv("ACE_MODEL", "us.anthropic.claude-haiku-4-5-20251001-v1:0")
client = LiteLLMClient(model=MODEL)

print(f"LLM client ready: {MODEL}")


LLM client ready: us.anthropic.claude-haiku-4-5-20251001-v1:0


 ## 4. Build Roles



 The three ACE roles share the same LLM client. Each is independently

 customisable (prompt templates, retries, etc.).

In [4]:
agent = Agent(client)
reflector = Reflector(client)
skill_manager = SkillManager(client)

print("Roles created: Agent, Reflector, SkillManager")


Roles created: Agent, Reflector, SkillManager


 ## 5. Define Training Samples

In [5]:
samples = [
    Sample(question="What is the capital of France?", ground_truth="Paris"),
    Sample(question="What is the capital of Japan?", ground_truth="Tokyo"),
    Sample(question="What is the capital of Brazil?", ground_truth="Brasilia"),
    Sample(question="What is the capital of Australia?", ground_truth="Canberra"),
    Sample(question="What is the capital of Nigeria?", ground_truth="Abuja"),
]

print(f"Prepared {len(samples)} training samples")


Prepared 5 training samples


 ---

 ## 6. ACE Runner — Full Adaptive Pipeline



 The `ACE` runner is the full closed-loop pipeline:

 ```

 Agent → Evaluate → Reflect → Tag → Update → Apply

 ```



 It takes `Sample` objects and an optional `TaskEnvironment`.

 ### 6a. With SimpleEnvironment



 `SimpleEnvironment` checks if the ground truth appears in the agent's

 answer (case-insensitive substring match).

In [6]:
skillbook = Skillbook()

ace = ACE.from_roles(
    agent=agent,
    reflector=reflector,
    skill_manager=skill_manager,
    environment=SimpleEnvironment(),
    skillbook=skillbook,
)

results = ace.run(samples[:3], epochs=1)

print(f"Processed {len(results)} samples\n")
for r in results:
    if r.error:
        print(f"  ERROR at {r.failed_at}: {r.error}")
    elif r.output:
        ctx: ACEStepContext = r.output
        answer = ctx.agent_output.final_answer if ctx.agent_output else "N/A"
        print(f"  Q: {r.sample.question}")
        print(f"  A: {answer}")


Processed 3 samples

  Q: What is the capital of France?
  A: The capital of France is Paris.
  Q: What is the capital of Japan?
  A: Tokyo is the capital of Japan.
  Q: What is the capital of Brazil?
  A: The capital of Brazil is Brasília.


In [7]:
print(f"\nSkillbook after 1 epoch:")
print(f"  Stats: {skillbook.stats()}")
for skill in skillbook.skills()[:5]:
    print(f"  - [{skill.id}] {skill.content}")



Skillbook after 1 epoch:
  Stats: {'sections': 5, 'skills': 7, 'tags': {'helpful': 7, 'harmful': 0, 'neutral': 0}}
  - [knowledge_retrieval-00001] Retrieve factual answers directly when no decomposition strategies apply
  - [answer_quality-00002] Include historical context or verification to strengthen factual answers
  - [problem_solving-00003] Assess skillbook before strategy selection to avoid unnecessary processing
  - [factual_recall-00004] Answer direct factual questions without strategic decomposition
  - [answer_quality-00005] Supplement factual answers with contextual details (location, dates, historical context)


 ### 6b. Custom Environment



 Create your own evaluator by subclassing `TaskEnvironment`.

In [8]:
class ExactMatchEnvironment(TaskEnvironment):
    """Strict evaluation: answer must exactly match ground truth."""

    def evaluate(self, sample: Sample, agent_output: AgentOutput) -> EnvironmentResult:
        expected = (sample.ground_truth or "").strip().lower()
        predicted = agent_output.final_answer.strip().lower()
        correct = expected == predicted

        return EnvironmentResult(
            feedback="Correct!" if correct else f"Wrong. Expected: {sample.ground_truth}",
            ground_truth=sample.ground_truth,
            metrics={"accuracy": 1.0 if correct else 0.0},
        )


print("ExactMatchEnvironment defined")

ExactMatchEnvironment defined


In [9]:
skillbook2 = Skillbook()

ace2 = ACE.from_roles(
    agent=Agent(client),
    reflector=Reflector(client),
    skill_manager=SkillManager(client),
    environment=ExactMatchEnvironment(),
    skillbook=skillbook2,
)

results2 = ace2.run(samples[:2], epochs=1)

for r in results2:
    if r.output:
        ctx = r.output
        print(f"  Q: {r.sample.question}")
        print(f"  A: {ctx.agent_output.final_answer if ctx.agent_output else 'N/A'}")
        if ctx.reflection:
            print(f"  Insight: {ctx.reflection.key_insight}")
        print()


  Q: What is the capital of France?
  A: The capital of France is Paris.
  Insight: This case represents a SUCCESS_CASE with a feedback system error. The model performed correctly; the contradiction lies in the evaluation layer, not the generation layer. Recommend investigating feedback pipeline integrity before attributing performance issues to model capability.

  Q: What is the capital of Japan?
  A: Tokyo is the capital of Japan.
  Insight: Model reasoning and factual knowledge were correct; the contradiction between correct prediction and negative feedback indicates a system-level evaluation error rather than model performance failure. This suggests need for feedback validation mechanisms.



In [10]:
print(f"\nSkillbook after 1 epoch:")
print(f"  Stats: {skillbook2.stats()}")
for skill in skillbook2.skills()[:5]:
    print(f"  - [{skill.id}] {skill.content}")



Skillbook after 1 epoch:
  Stats: {'sections': 0, 'skills': 0, 'tags': {'helpful': 0, 'harmful': 0, 'neutral': 0}}


 ### 6c. Without Environment



 When no environment is provided, `EvaluateStep` is a no-op. The Reflector

 still learns from ground-truth comparison in the trace.

In [15]:
skillbook3 = Skillbook()

ace3 = ACE.from_roles(
    agent=Agent(client),
    reflector=Reflector(client),
    skill_manager=SkillManager(client),
    skillbook=skillbook3,
    # No environment — EvaluateStep passes through
)

results3 = ace3.run(samples[:2], epochs=1)
print(f"Processed {len(results3)} samples (no environment)")
print(f"Skills learned: {skillbook3.stats()}")


16:43:07 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


INFO     [LiteLLM] 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


16:43:13 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler


INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler


16:43:13 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


...

INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler


16:43:32 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


INFO     [LiteLLM] 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


16:43:42 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler


INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler
Processed 2 samples (no environment)
Skills learned: {'sections': 3, 'skills': 6, 'tags': {'helpful': 6, 'harmful': 0, 'neutral': 0}}


 ### 6d. Multi-Epoch Training



 Multiple epochs let the agent revisit samples with an evolving skillbook.

 Skills accumulate and refine across passes.

In [16]:
skillbook4 = Skillbook()

ace4 = ACE.from_roles(
    agent=Agent(client),
    reflector=Reflector(client),
    skill_manager=SkillManager(client),
    environment=SimpleEnvironment(),
    skillbook=skillbook4,
)

results4 = ace4.run(samples, epochs=2)

print(f"Total results across 2 epochs: {len(results4)}")
print(f"Skills learned: {skillbook4.stats()}")

# Print per-epoch accuracy
for epoch in range(1, 3):
    epoch_results = [r for r in results4 if r.output and r.output.epoch == epoch]
    correct = sum(
        1 for r in epoch_results
        if r.output and r.output.agent_output
        and (r.sample.ground_truth or "").lower() in r.output.agent_output.final_answer.lower()
    )
    print(f"  Epoch {epoch}: {correct}/{len(epoch_results)} correct")


16:43:57 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


INFO     [LiteLLM] 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


16:44:04 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler


INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler


16:44:04 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


...

INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler


16:45:41 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


INFO     [LiteLLM] 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


16:45:52 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler


INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler
Total results across 2 epochs: 10
Skills learned: {'sections': 1, 'skills': 8, 'tags': {'helpful': 38, 'harmful': 1, 'neutral': 0}}
  Epoch 1: 4/5 correct
  Epoch 2: 4/5 correct


 ---

 ## 7. Manual Step-by-Step Pipeline



 Under the hood, runners compose `Pipeline` objects from individual steps.

 Here we build one by hand to see exactly what each step does.

In [17]:
from pipeline import Pipeline
from ace_next.steps import (
    AgentStep,
    EvaluateStep,
    ReflectStep,
    TagStep,
    UpdateStep,
    ApplyStep,
    learning_tail,
)

skillbook5 = Skillbook()
env = SimpleEnvironment()

# Build the full pipeline manually
pipe = Pipeline(
    [
        AgentStep(Agent(client)),
        EvaluateStep(env),
        *learning_tail(Reflector(client), SkillManager(client), skillbook5),
    ]
)

print(f"Pipeline steps: {len(pipe._steps)}")
print(f"  requires: {pipe.requires}")
print(f"  provides: {pipe.provides}")


Pipeline steps: 6
  requires: frozenset({'sample', 'skillbook'})
  provides: frozenset({'agent_output', 'skill_manager_output', 'reflection', 'trace'})


 ### Run a single sample through the manual pipeline

In [ ]:
sample = samples[0]

# Build the context the same way ACE._build_context() does
ctx = ACEStepContext(
    sample=sample,
    skillbook=SkillbookView(skillbook5),
    epoch=1,
    total_epochs=1,
    step_index=0,
    total_steps=1,
    global_sample_index=0,
)

print(f"Before pipeline:")
print(f"  Skills: {skillbook5.stats()}")
print(f"  agent_output: {ctx.agent_output}")

# Run the full pipeline on a single context
from pipeline.protocol import SampleResult

results_manual = pipe.run([ctx])

print(f"\nAfter pipeline:")
for r in results_manual:
    if r.error:
        print(f"  ERROR: {r.error}")
    elif r.output:
        out: ACEStepContext = r.output
        print(f"  Agent answer:      {out.agent_output.final_answer if out.agent_output else 'N/A'}")
        print(f"  Reflector insight:  {out.reflection.key_insight if out.reflection else 'N/A'}")
        print(f"  Skills now:        {skillbook5.stats()}")


16:58:04 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


Before pipeline:
  Skills: {'sections': 3, 'skills': 3, 'tags': {'helpful': 3, 'harmful': 0, 'neutral': 0}}
  agent_output: None
INFO     [LiteLLM] 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


16:58:11 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler


INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler


16:58:11 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


...

INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler


16:58:21 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


INFO     [LiteLLM] 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


16:58:31 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler


INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler


 ### Using `learning_tail()` as a building block



 `learning_tail()` returns the standard learning steps:

 `[ReflectStep, TagStep, UpdateStep, ApplyStep]` with optional

 deduplication and checkpoint steps appended.

In [20]:
skillbook6 = Skillbook()

tail = learning_tail(
    Reflector(client),
    SkillManager(client),
    skillbook6,
)

print(f"learning_tail() returns {len(tail)} steps:")
for step in tail:
    print(f"  - {type(step).__name__}")


learning_tail() returns 4 steps:
  - ReflectStep
  - TagStep
  - UpdateStep
  - ApplyStep


 ---

 ## 8. Checkpointing



 Save the skillbook every N successful samples so you can resume after

 interruption or compare skillbook evolution over time.

In [21]:
skillbook7 = Skillbook()

with tempfile.TemporaryDirectory() as tmpdir:
    ace7 = ACE.from_roles(
        agent=Agent(client),
        reflector=Reflector(client),
        skill_manager=SkillManager(client),
        environment=SimpleEnvironment(),
        skillbook=skillbook7,
        checkpoint_dir=tmpdir,
        checkpoint_interval=2,  # save every 2 successful samples
    )

    results7 = ace7.run(samples, epochs=1)

    saved = sorted(Path(tmpdir).glob("*.json"))
    print("Checkpoint files:")
    for f in saved:
        print(f"  {f.name}  ({f.stat().st_size} bytes)")


16:59:25 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


INFO     [LiteLLM] 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


16:59:31 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler


INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler


16:59:31 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


...

INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler


17:00:17 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


INFO     [LiteLLM] 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock
INFO     [ace_next.steps.checkpoint] CheckpointStep: saved checkpoint at sample 4 → /tmp/tmpqnq54gga/checkpoint_4.json


17:00:28 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler


INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler
Checkpoint files:
  checkpoint_2.json  (2464 bytes)
  checkpoint_4.json  (4971 bytes)
  latest.json  (4971 bytes)


 ---

 ## 9. Deduplication



 Merge near-duplicate skills to keep the skillbook compact. The

 `DeduplicationManager` runs periodically during training.

In [22]:
from ace_next import DeduplicationManager, SimilarityDetector
from ace_next.protocols import DeduplicationConfig

skillbook8 = Skillbook()

dedup = DeduplicationManager(
    DeduplicationConfig(similarity_threshold=0.85)
)

ace8 = ACE.from_roles(
    agent=Agent(client),
    reflector=Reflector(client),
    skill_manager=SkillManager(client),
    environment=SimpleEnvironment(),
    skillbook=skillbook8,
    dedup_manager=dedup,
    dedup_interval=3,  # run dedup every 3 samples
)

results8 = ace8.run(samples, epochs=1)

print(f"Skills after training with dedup: {skillbook8.stats()}")


17:08:53 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


INFO     [LiteLLM] 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


17:08:59 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler


INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler


17:08:59 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


...

INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler


17:09:42 - LiteLLM:INFO: utils.py:3889 - 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


INFO     [LiteLLM] 
LiteLLM completion() model= us.anthropic.claude-haiku-4-5-20251001-v1:0; provider = bedrock


17:09:54 - LiteLLM:INFO: utils.py:1629 - Wrapper: Completed Call, calling success_handler


INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler
Skills after training with dedup: {'sections': 3, 'skills': 7, 'tags': {'helpful': 8, 'harmful': 0, 'neutral': 0}}


 ---

 ## 10. Observability with Opik



 `OpikStep` is an explicit, opt-in pipeline step that logs traces to Opik.

 It is **not** wired into `learning_tail()` — you append it yourself.



 Three usage patterns:

 1. **Pipeline traces + LLM cost tracking** — append `OpikStep()` (default)

 2. **Pipeline traces only** — `OpikStep(register_litellm_callback=False)`

 3. **LLM cost tracking only** — `register_opik_litellm_callback()` (no step)

In [24]:
from ace_next import OpikStep, OPIK_AVAILABLE, register_opik_litellm_callback

print(f"Opik available: {OPIK_AVAILABLE}")


Opik available: True


 ### 10a. Append OpikStep to a custom pipeline



 Place it at the end — after the learning tail.

In [25]:
if OPIK_AVAILABLE:
    skillbook_opik = Skillbook()

    pipe_with_opik = Pipeline(
        [
            AgentStep(Agent(client)),
            EvaluateStep(SimpleEnvironment()),
            *learning_tail(Reflector(client), SkillManager(client), skillbook_opik),
            OpikStep(project_name="ace-demo"),
        ]
    )
    print(f"Pipeline steps (with Opik): {len(pipe_with_opik._steps)}")
    for step in pipe_with_opik._steps:
        print(f"  - {type(step).__name__}")
else:
    print("Opik not installed — skipping pipeline example")


Pipeline steps (with Opik): 7
  - AgentStep
  - EvaluateStep
  - ReflectStep
  - TagStep
  - UpdateStep
  - ApplyStep
  - OpikStep


 ### 10b. LLM-level cost tracking only



 If you only want per-LLM-call token/cost logging without pipeline traces,

 use the standalone helper. This registers an `OpikLogger` callback on

 `litellm.callbacks`.

In [26]:
if OPIK_AVAILABLE:
    registered = register_opik_litellm_callback(project_name="ace-demo")
    print(f"LiteLLM Opik callback registered: {registered}")
else:
    print("Opik not installed — skipping callback example")


LiteLLM Opik callback registered: True


 ---

 ## 11. Skillbook Persistence — Save & Reload



 Save the learned skillbook to disk and reload it in a future session.

In [11]:
with tempfile.TemporaryDirectory() as tmpdir:
    path = Path(tmpdir) / "learned_skillbook.json"

    # Save
    skillbook.save_to_file(str(path))
    print(f"Saved to {path.name}  ({path.stat().st_size} bytes)")

    # Reload
    reloaded = Skillbook.load_from_file(str(path))
    print(f"Reloaded: {reloaded.stats()}")
    print(f"Stats match: {reloaded.stats() == skillbook.stats()}")


Saved to learned_skillbook.json  (5561 bytes)
Reloaded: {'sections': 5, 'skills': 7, 'tags': {'helpful': 7, 'harmful': 0, 'neutral': 0}}
Stats match: True


 ---

 ## 12. TraceAnalyser — Learning from Pre-Recorded Traces



 `TraceAnalyser` runs the learning tail only — no Agent, no Evaluate.

 Feed it raw trace dicts (the same shape ReflectStep expects) and it

 builds a skillbook from historical data.

In [12]:
# Simulate some pre-recorded traces (e.g., from browser-use history logs)
traces = [
    {
        "question": "Book a flight from NYC to London",
        "reasoning": "Step 1: Opened booking site. Step 2: Searched flights. Step 3: Selected cheapest option.",
        "answer": "Booked flight AA100 for $450",
        "skill_ids": [],
        "feedback": "Task succeeded in 3 steps",
        "ground_truth": None,
    },
    {
        "question": "Find the cheapest hotel in Paris",
        "reasoning": "Step 1: Opened hotel site. Step 2: Set filters. Step 3: Sorted by price. Step 4: Cookie popup blocked view.",
        "answer": "Failed: could not dismiss cookie popup",
        "skill_ids": [],
        "feedback": "Task failed — cookie popup blocked interaction after step 3",
        "ground_truth": None,
    },
    {
        "question": "Check weather in Tokyo",
        "reasoning": "Step 1: Navigated to weather.com. Step 2: Searched Tokyo. Step 3: Read forecast.",
        "answer": "Tokyo: 22C, partly cloudy",
        "skill_ids": [],
        "feedback": "Task succeeded in 3 steps — fast and accurate",
        "ground_truth": None,
    },
]

skillbook9 = Skillbook()

analyser = TraceAnalyser.from_roles(
    reflector=Reflector(client),
    skill_manager=SkillManager(client),
    skillbook=skillbook9,
)

results9 = analyser.run(traces, epochs=1)

print(f"Analysed {len(results9)} traces")
print(f"Skills learned: {skillbook9.stats()}")
for skill in skillbook9.skills()[:5]:
    print(f"  - [{skill.section}] {skill.content}")


Analysed 3 traces
Skills learned: {'sections': 2, 'skills': 6, 'tags': {'helpful': 6, 'harmful': 0, 'neutral': 0}}
  - [information_retrieval] Use weather.com for direct weather information queries
  - [information_retrieval] Navigate tool → search location → extract data for geographic queries
  - [information_retrieval] Extract quantified data with specific values and conditions
  - [web_ui_patterns] Identify and dismiss cookie consent popups before accessing page content
  - [web_ui_patterns] Handle unexpected UI overlays appearing mid-interaction during web tasks


 ### Multi-epoch trace analysis



 Each epoch re-processes all traces with the evolving skillbook.

 Early epochs extract obvious patterns; later epochs refine.

In [13]:
skillbook10 = Skillbook()

analyser2 = TraceAnalyser.from_roles(
    reflector=Reflector(client),
    skill_manager=SkillManager(client),
    skillbook=skillbook10,
)

results10 = analyser2.run(traces, epochs=2)

print(f"Total results across 2 epochs: {len(results10)}")
print(f"Skills after 2 epochs: {skillbook10.stats()}")


Total results across 2 epochs: 6
Skills after 2 epochs: {'sections': 2, 'skills': 6, 'tags': {'helpful': 9, 'harmful': 0, 'neutral': 0}}


 ---

 ## 13. Mixed Workflow — TraceAnalyser then ACE



 A common pattern: build an initial skillbook from historical traces,

 then deploy with live learning.

In [14]:
# Phase 1: Build skillbook from historical data
shared_skillbook = Skillbook()

analyser_phase1 = TraceAnalyser.from_roles(
    reflector=Reflector(client),
    skill_manager=SkillManager(client),
    skillbook=shared_skillbook,
)
analyser_phase1.run(traces, epochs=1)

print(f"Phase 1 — TraceAnalyser:")
print(f"  Skills from traces: {shared_skillbook.stats()}")

# Phase 2: Deploy with live ACE learning (reuse the evolved skillbook)
ace_phase2 = ACE.from_roles(
    agent=Agent(client),
    reflector=Reflector(client),
    skill_manager=SkillManager(client),
    environment=SimpleEnvironment(),
    skillbook=shared_skillbook,
)

results_phase2 = ace_phase2.run(samples[:3], epochs=1)

print(f"\nPhase 2 — ACE live learning:")
print(f"  Processed {len(results_phase2)} samples")
print(f"  Skills after live learning: {shared_skillbook.stats()}")


Phase 1 — TraceAnalyser:
  Skills from traces: {'sections': 3, 'skills': 9, 'tags': {'helpful': 9, 'harmful': 0, 'neutral': 0}}

Phase 2 — ACE live learning:
  Processed 3 samples
  Skills after live learning: {'sections': 3, 'skills': 12, 'tags': {'helpful': 17, 'harmful': 0, 'neutral': 7}}


 ---

 ## 14. Error Handling



 Failed samples are captured in `SampleResult.error` — the pipeline

 never drops a sample silently. Other samples continue processing.

In [15]:
bad_samples = [
    samples[0],
    Sample(question="", ground_truth=""),  # edge case: empty question
    samples[1],
]

skillbook11 = Skillbook()
ace11 = ACE.from_roles(
    agent=Agent(client),
    reflector=Reflector(client),
    skill_manager=SkillManager(client),
    environment=SimpleEnvironment(),
    skillbook=skillbook11,
)

results11 = ace11.run(bad_samples, epochs=1)

for i, r in enumerate(results11, 1):
    status = "OK" if r.error is None else f"FAIL ({r.failed_at})"
    if r.output and r.output.agent_output:
        answer = r.output.agent_output.final_answer
    else:
        answer = "N/A"
    print(f"  [{i}] {status:20s}  answer={answer}")


  [1] OK                    answer=The capital of France is Paris.
  [2] OK                    answer=No problem to solve. This prompt provides the operational framework and instructions for ACE Agent v2.1, but does not contain a specific question or problem in the 'Question' field. To proceed, please provide: (1) A specific question or problem to solve, (2) Relevant skillbook entries with strategy IDs and content, and (3) Any additional context needed. Once a question is provided, I will apply the skillbook protocol with complete step-by-step reasoning and specific skill citations.
  [3] OK                    answer=Tokyo is the capital of Japan.


 ---

 ## 15. Inspecting the SkillbookView



 Steps receive a read-only `SkillbookView` on the context.

 This prevents accidental mutations from within pipeline steps.

In [16]:
sb = Skillbook()
view = SkillbookView(sb)

print(f"SkillbookView: {view}")
print(f"  len:    {len(view)}")
print(f"  stats:  {view.stats()}")
print(f"  prompt: {view.as_prompt()[:200]}...")

# Iterate over skills in the view
for skill in view:
    print(f"  - {skill.id}: {skill.content}")


SkillbookView: SkillbookView(0 skills)
  len:    0
  stats:  {'sections': 0, 'skills': 0, 'tags': {'helpful': 0, 'harmful': 0, 'neutral': 0}}
  prompt: skills[0	]:...


 ---

 ## Summary



 | What | How |

 |------|-----|

 | Full pipeline | `ACE.from_roles(agent=..., reflector=..., skill_manager=...)` |

 | With environment | `ACE.from_roles(..., environment=SimpleEnvironment())` |

 | Without environment | `ACE.from_roles(...)` — EvaluateStep is a no-op |

 | Multi-epoch | `ace.run(samples, epochs=3)` |

 | Checkpointing | `ACE.from_roles(..., checkpoint_dir="./ckpts", checkpoint_interval=10)` |

 | Deduplication | `ACE.from_roles(..., dedup_manager=dedup, dedup_interval=5)` |

 | Opik tracing | `Pipeline([...steps..., OpikStep(project_name="my-project")])` |

 | LLM cost tracking | `register_opik_litellm_callback()` |

 | Trace analysis | `TraceAnalyser.from_roles(reflector=..., skill_manager=...)` |

 | Save skillbook | `ace.save("path.json")` or `skillbook.save_to_file("path.json")` |

 | Load skillbook | `Skillbook.from_file("path.json")` |

 | Manual steps | `Pipeline([AgentStep(a), EvaluateStep(e), *learning_tail(r, sm, sb)])` |

 | Learning tail | `learning_tail(reflector, skill_manager, skillbook)` |



 **Pipeline:**

 ```

 ACE:            Agent → Evaluate → Reflect → Tag → Update → Apply → [Dedup] → [Checkpoint] → [Opik]

 TraceAnalyser:                     Reflect → Tag → Update → Apply → [Dedup] → [Checkpoint] → [Opik]

 ```